# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import keras
import vis
import pandas as pd

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout
from keras import backend as K

tf.set_random_seed(42)

Using TensorFlow backend.


In [0]:
(trainX, trainY),(testX, testY) = tf.keras.datasets.cifar10.load_data()

In [3]:
print(trainX.shape)
print(testX.shape)
print(trainY.shape)
print(testY.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 1)
(10000, 1)


In [0]:
trainY = trainY.reshape(50000)
testY = testY.reshape(10000)

In [0]:
trainX_lt5 = trainX[trainY < 5]
trainY_lt5 = trainY[trainY < 5]
testX_lt5 = testX[testY < 5]
testY_lt5 = testY[testY < 5]

In [6]:
print(trainX_lt5.shape)
print(testX_lt5.shape)
print(trainY_lt5.shape)
print(testY_lt5.shape)

(25000, 32, 32, 3)
(5000, 32, 32, 3)
(25000,)
(5000,)


### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

*We will only start work with the 0-4 Category Dataset*

In [0]:
trainX_lt5_conv = trainX_lt5.reshape(trainX_lt5.shape[0], 32, 32, 3)
testX_lt5_conv = testX_lt5.reshape(testX_lt5.shape[0], 32, 32, 3)
input_shape = (32, 32, 3)

In [0]:
trainX_lt5_conv =  trainX_lt5_conv.astype("float32") / 255
testX_lt5_conv = testX_lt5_conv.astype("float32") / 255

In [0]:
trainY_lt5_class = keras.utils.to_categorical(trainY_lt5, 5)
testY_lt5_class = keras.utils.to_categorical(testY_lt5, 5)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
model_lt5 = Sequential()
model_lt5.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape,name='conv_1'))
model_lt5.add(MaxPooling2D(pool_size=(2, 2),name='max_1'))
model_lt5.add(Conv2D(128, (3, 3), activation='relu',name='conv_2'))
model_lt5.add(MaxPooling2D(pool_size=(2, 2),name='max_2'))
model_lt5.add(Conv2D(128, (3, 3), activation='relu',name='conv_3'))
model_lt5.add(MaxPooling2D(pool_size=(2, 2),name='max_3'))
model_lt5.add(Dropout(0.25,name='drop_1'))
model_lt5.add(Flatten())
model_lt5.add(Dense(128, activation='relu',name='dense_1'))
model_lt5.add(Dropout(0.5, name='drop_2'))
model_lt5.add(Dense(5, activation='softmax',name='dense_2'))

In [0]:
model_lt5.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [16]:
%%time 
output_04 = model_lt5.fit(trainX_lt5_conv, trainY_lt5_class, batch_size=128, epochs=5, verbose=1,
                    validation_data=(testX_lt5_conv, testY_lt5_class))

Train on 25000 samples, validate on 5000 samples
Epoch 1/5
25000/25000 [==============================] - 71s 3ms/step - loss: 1.2761 - acc: 0.4442 - val_loss: 1.0012 - val_acc: 0.5860
Epoch 2/5
25000/25000 [==============================] - 70s 3ms/step - loss: 0.9966 - acc: 0.5900 - val_loss: 0.8787 - val_acc: 0.6414
Epoch 3/5
25000/25000 [==============================] - 70s 3ms/step - loss: 0.8967 - acc: 0.6424 - val_loss: 0.8583 - val_acc: 0.6500
Epoch 4/5
25000/25000 [==============================] - 72s 3ms/step - loss: 0.8248 - acc: 0.6765 - val_loss: 0.7578 - val_acc: 0.7088
Epoch 5/5
25000/25000 [==============================] - 70s 3ms/step - loss: 0.7614 - acc: 0.7045 - val_loss: 0.7094 - val_acc: 0.7296
CPU times: user 11min 1s, sys: 12 s, total: 11min 13s
Wall time: 5min 53s


### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [17]:
#Freezing layers in the model which don't have 'dense' in their name
for layer in model_lt5.layers:
  if('dense' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False

#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in model_lt5.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

conv_1
False
max_1
False
conv_2
False
max_2
False
conv_3
False
max_3
False
drop_1
False
flatten_2
False
dense_1
True
drop_2
False
dense_2
True


### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:
trainX_gt5 = trainX[trainY >= 5]
trainY_gt5 = trainY[trainY >= 5] - 5
testX_gt5 = testX[testY >= 5]
testY_gt5 = testY[testY >= 5] - 5

In [0]:
trainX_gt5_conv = trainX_gt5.reshape(trainX_gt5.shape[0], 32, 32, 3)
testX_gt5_conv = testX_gt5.reshape(testX_gt5.shape[0], 32, 32, 3)

In [0]:
trainX_gt5_conv =  trainX_gt5_conv.astype("float32") / 255
testX_gt5_conv = testX_gt5_conv.astype("float32") / 255

In [0]:
trainY_gt5_class = keras.utils.to_categorical(trainY_gt5, 5)
testY_gt5_class = keras.utils.to_categorical(testY_gt5, 5)

In [22]:
%%time 
output_05 = model_lt5.fit(trainX_gt5_conv, trainY_gt5_class, batch_size=512, epochs=5, verbose=1,
                    validation_data=(testX_gt5_conv, testY_gt5_class))

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Train on 25000 samples, validate on 5000 samples
Epoch 1/5
25000/25000 [==============================] - 73s 3ms/step - loss: 1.5111 - acc: 0.3733 - val_loss: 1.0450 - val_acc: 0.5966
Epoch 2/5
25000/25000 [==============================] - 73s 3ms/step - loss: 1.0759 - acc: 0.5717 - val_loss: 0.9692 - val_acc: 0.6326
Epoch 3/5
25000/25000 [==============================] - 74s 3ms/step - loss: 0.9821 - acc: 0.6171 - val_loss: 0.8326 - val_acc: 0.6814
Epoch 4/5
25000/25000 [==============================] - 74s 3ms/step - loss: 0.9207 - acc: 0.6440 - val_loss: 0.7747 - val_acc: 0.7050
Epoch 5/5
25000/25000 [==============================] - 73s 3ms/step - loss: 0.8703 - acc: 0.6641 - val_loss: 0.7404 - val_acc: 0.7154
CPU times: user 11min 39s, sys: 9.2 s, total: 11min 49s
Wall time: 6min 6s


# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [0]:
twenty_train.target

In [0]:
twenty_train.target_names

In [0]:
twenty_train.data[0:5]

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score